In [2]:
import warnings

import pandas as pd
import numpy as np
import pickle
import operator
import re
import gc
import gensim

import json

In [3]:
# TM_MODELS_PATH = '/media/pablo/data/Tesis/models/old/tm_feats/'
from os.path import join

In [4]:
# Leer todos los posts de los 3 candidatos

In [15]:
posts_text = []

In [18]:
for cand in ['hectorbaldassi', 'martinllaryoraoficial', 'pablocarrook']:
    df = pd.read_csv('/home/pablo/GDrive/BigSocialData/facebook/%s/posts.csv' % cand)
    posts_text += list(df.message[df.message.notnull()].values)

In [19]:
len(posts_text)

1519

In [20]:
corpus = posts_text

In [21]:
def preprocess(doc):
    pre_doc = doc
        
    # remover URLs
    pre_doc = re.sub(
        r"https?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",
        " ", pre_doc)
    
    # minúsculas
    pre_doc = pre_doc.lower()

    # volar acentos
    pre_doc = gensim.utils.deaccent(pre_doc)

    # remove bullshit
    pre_doc = re.sub(r"\@|\'|\"|\\|…|\/|\-|\||\(|\)|\.|\,|\!|\?|\:|\;|“|”|’|—", " ", pre_doc)
    
    # contraer vocales
    for v in 'aeiou':
        pre_doc = re.sub(r"[%s]+" % v, v, pre_doc)    
    
    # normalizar espacio en blanco
    pre_doc = re.sub(r"\s+", " ", pre_doc)
    pre_doc = re.sub(r"(^\s)|(\s$)", "", pre_doc)
    
    return pre_doc

In [22]:
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.data import load
from nltk.stem import SnowballStemmer
from string import punctuation

spanish_tokenizer = load('tokenizers/punkt/spanish.pickle')

# stopwords en español
spanish_stopwords = stopwords.words('spanish')

# spanish stemmer
stemmer = SnowballStemmer('spanish')

# punctuation to remove
non_words = list(punctuation)

# we add spanish punctuation
non_words.extend(['¿', '¡'])
non_words.extend(map(str, range(10)))

stemmer = SnowballStemmer('spanish')

def trystem(t):
    try:
        t = stemmer.stem(t)
    except Exception:
        pass
    return t

def tokenize(text, stem=True, remove_stopwords=False):
    text = text.lower()
    result = []
    
    for sentence in spanish_tokenizer.tokenize(text):
        # remover puntuación
        text = ''.join([c for c in sentence if c not in non_words])
        
        # tokenize
        tokens = word_tokenize(text)

        if remove_stopwords:
            tokens = [t for t in tokens if t not in spanish_stopwords]

        # tokens de al menos 2 letras
        tokens = [t for t in tokens if len(t) > 1]
            
        # stem
        if stem:
            tokens = [trystem(t) for t in tokens]
        
        result += tokens
        
    return result

In [23]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [24]:
class get_docs(object):
    def __init__(self, corpus):
        self.corpus = corpus

    def __iter__(self):
        for doc in self.corpus:
            tokens = tokenize(preprocess(doc), remove_stopwords=True)
            yield tokens

In [20]:
# from gensim.models.phrases import Phraser, Phrases

In [21]:
# phrases = Phrases(get_docs(corpus), min_count=5)
# bigram = Phraser(phrases)
# trigram = Phrases(bigram[get_docs(corpus)], min_count=5)
# dictionary = gensim.corpora.Dictionary(trigram[get_docs(corpus)])

In [25]:
dictionary = gensim.corpora.Dictionary(get_docs(corpus))
dictionary.filter_extremes(no_below=5, no_above=0.3, keep_n=None)

In [26]:
sorted(dictionary.token2id.items(), key=lambda x:-x[1])[:50]

[(u'podr', 1090),
 (u'convert', 1089),
 (u'cristin', 1088),
 (u'agu', 1087),
 (u'rural', 1086),
 (u'salv', 1085),
 (u'socied', 1084),
 (u'hinch', 1083),
 (u'brocher', 1082),
 (u'cruz', 1081),
 (u'matriz', 1080),
 (u'empuj', 1079),
 (u'adecu', 1078),
 (u'gan', 1077),
 (u'neces', 1076),
 (u'obras', 1075),
 (u'junt', 1074),
 (u'festej', 1073),
 (u'fueg', 1072),
 (u'labaqu', 1071),
 (u'\xa1graci', 1070),
 (u'escuel', 1069),
 (u'intens', 1068),
 (u'paisaj', 1067),
 (u'vecin', 1066),
 (u'teatr', 1065),
 (u'benefici', 1064),
 (u'gat', 1063),
 (u'conect', 1062),
 (u'excelent', 1061),
 (u'cienci', 1060),
 (u'kv', 1059),
 (u'futur', 1058),
 (u'tio', 1057),
 (u'tard', 1056),
 (u'tare', 1055),
 (u'\ud83c\udde6\ud83c\uddf7', 1054),
 (u'tip', 1053),
 (u'diput', 1052),
 (u'hector', 1051),
 (u'mientr', 1050),
 (u'cuart', 1049),
 (u'tranquil', 1048),
 (u'pobrez', 1047),
 (u'paraguay', 1046),
 (u'cultur', 1045),
 (u'mejor', 1044),
 (u'rufeil', 1043),
 (u'exist', 1042),
 (u'ar', 1041)]

In [27]:
len(dictionary.dfs)

1091

In [ ]:
# dictionary.save(join(TM_MODELS_PATH, "tweets_es.dict"))

In [9]:
# dictionary = gensim.corpora.Dictionary.load(join(TM_MODELS_PATH, "tweets_es.dict"))

In [28]:
bow = [dictionary.doc2bow(doc) for doc in get_docs(corpus)]

In [25]:
# with open(join(TM_MODELS_PATH,'tweets_es_bow.pickle'), 'wb') as f:
#     pickle.dump(bow,f)

In [48]:
from math import ceil
iters = 100
passes = 10
workers = 7
n_topics = 20
chunksize = int(ceil(len(bow)*1.0/workers))

In [49]:
model = gensim.models.LdaMulticore(
        corpus=bow,
        id2word=dictionary,
        num_topics=n_topics,
        iterations=iters,
        alpha=0.001,
        passes=passes,
        chunksize=chunksize,
        workers=workers
)

In [50]:
import pyLDAvis.gensim
import pyLDAvis

In [51]:
viz = pyLDAvis.gensim.prepare(model, [v for v in bow], model.id2word)

In [52]:
pyLDAvis.display(viz)